<center><h2>Jane Street Market Prediction | MLP Baseline | katsu1110 </h2></center><hr>

The idea is:

- Neural Network (e.g. multi-layer perceptron) seems to perform well in this competition.
- In most of public notebooks, records with weight=0 is ignored.
- How can we use those 0-weight records? One way with NN would be to **pre-train**.
- So the total procedure is (1) pretrain NN with records with weight=0 (2) train NN with records with weight > 0 (3) inference

This notebook is for **stage 2 (train)**.

For other stages, see:

- [[JaneStreet] MLP Pretrain (stage1: weight=0)](https://www.kaggle.com/code1110/janestreet-mlp-pretrain-stage1-weight-0/notebook)
- [[JaneStreet] MLP inference (stage3)](https://www.kaggle.com/code1110/janestreet-mlp-inference-stage3)

This notebook loads feathered-data from [my another notebook](https://www.kaggle.com/code1110/janestreet-save-as-feather?scriptVersionId=47635784) such that we don't have to spend our time on waiting long for loading csv files.

In this notebook we treat the task as a binary classification.

In [ ]:
import janestreet
import numpy as np
import pandas as pd

import os, sys
import gc
import math
import random
import pathlib
from tqdm import tqdm
from typing import List, NoReturn, Union, Tuple, Optional, Text, Generic, Callable, Dict
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer
from tqdm import tqdm_notebook as tqdm
from typing import List, NoReturn, Union, Tuple, Optional, Text, Generic, Callable, Dict

# keras
import tensorflow as tf
import tensorflow_addons as tfa

# visualize
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from matplotlib_venn import venn2
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter
sns.set_context("talk")
style.use('fivethirtyeight')
pd.options.display.max_columns = None

import warnings
warnings.filterwarnings('ignore')

# Config
Some configuration setups.

In [ ]:
# tf setup
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

MIXED_PRECISION = False
XLA_ACCELERATE = True

if MIXED_PRECISION:
    from tensorflow.keras.mixed_precision import experimental as mixed_precision
    if tpu: policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
    else: policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')


In [ ]:
DEBUG = False

SEED = 2021 # Happy new year
INPUT_DIR = '../input/janestreet-save-as-feather/'
TRADING_THRESHOLD = 0.50 # 0 ~ 1: The smaller, the more aggressive
DATE_BEGIN = 86 # 0 ~ 499: set 0 for model training using the complete data 
CV_STRATEGY = 'PurgedGroupTimeSeriesSplit' # StratifiedGroupKFold, GroupKFold, PurgedGroupTimeSeriesSplit
FOLDS = 5 # number of folds
THRESHOLD = 0.5 # action threshold: the larger value, the more conservative

STAGE = 2 # pretrain (1), train (2), inference (3)

if STAGE == 2:
    WEIGHT_DIR = '../input/janestreet-mlp-pretrain-stage1-weight-0/'
elif STAGE == 3:
    WEIGHT_DIR = '../input/janestreet-mlp-train-stage2-weight-0/'

# CV Strategy

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import random
from collections import Counter, defaultdict
from sklearn import model_selection

# ---- GroupKFold ----
class GroupKFold(object):
    """
    GroupKFold with random shuffle with a sklearn-like structure
    """

    def __init__(self, n_splits=4, shuffle=True, random_state=42):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state

    def get_n_splits(self, X=None, y=None, group=None):
        return self.n_splits

    def split(self, X, y, group):
        kf = model_selection.KFold(n_splits=self.n_splits, shuffle=self.shuffle, random_state=self.random_state)
        unique_ids = X[group].unique()
        for fold, (tr_group_idx, va_group_idx) in enumerate(kf.split(unique_ids)):
            # split group
            tr_group, va_group = unique_ids[tr_group_idx], unique_ids[va_group_idx]
            train_idx = np.where(X[group].isin(tr_group))[0]
            val_idx = np.where(X[group].isin(va_group))[0]
            yield train_idx, val_idx

# ---- StratifiedGroupKFold ----
class StratifiedGroupKFold(object):
    """
    StratifiedGroupKFold with random shuffle with a sklearn-like structure
    """

    def __init__(self, n_splits=4, shuffle=True, random_state=42):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state

    def get_n_splits(self, X=None, y=None, group=None):
        return self.n_splits

    def split(self, X, y, group):
        labels_num = np.max(y) + 1
        y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
        y_distr = Counter()
        groups = X[group].values
        for label, g in zip(y, groups):
            y_counts_per_group[g][label] += 1
            y_distr[label] += 1

        y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
        groups_per_fold = defaultdict(set)

        def eval_y_counts_per_fold(y_counts, fold):
            y_counts_per_fold[fold] += y_counts
            std_per_label = []
            for label in range(labels_num):
                label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(self.n_splits)])
                std_per_label.append(label_std)
            y_counts_per_fold[fold] -= y_counts
            return np.mean(std_per_label)
        
        groups_and_y_counts = list(y_counts_per_group.items())
        random.Random(self.random_state).shuffle(groups_and_y_counts)

        for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
            best_fold = None
            min_eval = None
            for i in range(self.n_splits):
                fold_eval = eval_y_counts_per_fold(y_counts, i)
                if min_eval is None or fold_eval < min_eval:
                    min_eval = fold_eval
                    best_fold = i
            y_counts_per_fold[best_fold] += y_counts
            groups_per_fold[best_fold].add(g)

        all_groups = set(groups)
        for i in range(self.n_splits):
            train_groups = all_groups - groups_per_fold[i]
            test_groups = groups_per_fold[i]

            train_idx = [i for i, g in enumerate(groups) if g in train_groups]
            test_idx = [i for i, g in enumerate(groups) if g in test_groups]

            yield train_idx, test_idx


# Load data
I have already saved the training data in the feather-format in [my another notebook](https://www.kaggle.com/code1110/janestreet-save-as-feather?scriptVersionId=47635784). Loading csv takes time but loading feather is really light:)

In [ ]:
os.listdir(INPUT_DIR)

In [ ]:
train = pd.read_feather('../input/janestreet-save-as-feather/train.feather') # faster data load

In [ ]:
# remove weight = 0 for saving memory 
original_size = train.shape[0]

# use data later than DATE_BEGIN
train = train.query(f'date >= {DATE_BEGIN}')

# training stage
if STAGE == 1:
    train = train.query('weight == 0').reset_index(drop=True)
elif STAGE == 2:
    train = train.query('weight > 0').reset_index(drop=True)
elif STAGE == 3:
    print('inference')

print('Train size reduced from {:,} to {:,}.'.format(original_size, train.shape[0]))

In [ ]:
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use
train.fillna(train.mean(),inplace=True)
train['action'] =  (  (train['resp_1'] > 0 ) & (train['resp_2'] > 0 ) & (train['resp_3'] > 0 ) & (train['resp_4'] > 0 ) &  (train['resp'] > 0 )   ).astype('int')
features = [c for c in train.columns if 'feature' in c]

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

In [ ]:
if DEBUG:
    train = train.sample(10000, random_state=SEED)

X = train[features].values
y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T #Multitarget

f_mean = np.mean(train[features[1:]].values,axis=0)

# NN

In [ ]:
def seed_everything(seed : int) -> NoReturn :    
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

In [ ]:
nn_params = {
    'hidden_unit': 256,
    'num_layers': 3,
    'gaussnoise': 0.01,
    'dropout': 0.2,
    'learning_rate': 1e-03,
    'label_smoothing': 1e-02,
    'batch_size': 1024,
    'epochs': 196
}

def create_model(nn_params, input_dim, output_dim):
    # input
    inputs = tf.keras.layers.Input(input_dim)
    
    # normalize
    x = tf.keras.layers.BatchNormalization()(inputs)
    
    # ffn
    for i in range(nn_params['num_layers']):
        x = tf.keras.layers.Dense(nn_params['hidden_unit'] // (2 ** i), activation='relu')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.GaussianNoise(nn_params['gaussnoise'])(x)
        x = tf.keras.layers.Dropout(nn_params['dropout'])(x)
    x = tf.keras.layers.Dense(output_dim, activation='sigmoid')(x)
    
    model = tf.keras.models.Model(inputs=inputs, outputs=x)
    
    # compile
    opt = tf.keras.optimizers.Adam(learning_rate=nn_params['learning_rate'])
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=nn_params['label_smoothing'])
    model.compile(optimizer=opt, 
                  loss=loss, 
                  metrics=[tf.keras.metrics.AUC(name = 'auc')])
    return model

model = create_model(nn_params, 130, 5)
model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

# Model Fitting

In [ ]:
%%time

if STAGE == 3: # simply load NN weights for inference
    models = []
    
    for fold in tqdm(range(FOLDS)):
        tf.keras.backend.clear_session()
        model = create_model(nn_params, X.shape[-1], y.shape[-1])
        model.load_weights(WEIGHT_DIR + f'model_{SEED}_{fold}.hdf5')
        models.append(model)
    
else: # fit
    
    if CV_STRATEGY == 'PurgedGroupTimeSeriesSplit':
        gkf = PurgedGroupTimeSeriesSplit(n_splits=FOLDS, group_gap=20)
        splits = list(gkf.split(y, groups=train['date'].values))    

    elif CV_STRATEGY == "GroupKFold":
        cv = GroupKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
        splits = cv.split(train, train['resp'].values.astype(int), 'date')

    elif CV_STRATEGY ==  "StratifiedGroupKFold":
        cv = StratifiedGroupKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
        splits = cv.split(train, train['resp'].values.astype(int), 'date')

    models = []
    for fold, (train_indices, test_indices) in tqdm(enumerate(splits)):
        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]

        # model
        tf.keras.backend.clear_session()
        model = create_model(nn_params, X.shape[-1], y.shape[-1])

        if STAGE == 2:
            model.load_weights(WEIGHT_DIR + f'model_{SEED}_{fold}.hdf5')

        # callbacks
        er = tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True, monitor='val_loss')
        ReduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, verbose=1, mode='min')
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=f'./model_{SEED}_{fold}.hdf5', 
                                                                       save_weights_only=True, verbose=0, monitor='val_loss', save_best_only=True)
        nn_callbacks = [er, ReduceLR, model_checkpoint_callback]

        # fit
        model.fit(X_train, y_train, validation_data=(X_test,y_test), 
                  epochs=nn_params['epochs'], batch_size=nn_params['batch_size'], callbacks=nn_callbacks)
        models.append(model)

# Submit

In [ ]:
if STAGE == 3:
    f = np.median

    models = models[-3:]

    import janestreet
    env = janestreet.make_env()
    for (test_df, pred_df) in tqdm(env.iter_test()):
        if test_df['weight'].item() > 0:
            x_tt = test_df.loc[:, features].values
            if np.isnan(x_tt[:, 1:].sum()):
                x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
            pred = np.mean([model(x_tt, training = False).numpy() for model in models],axis=0)
            pred = f(pred)
            pred_df.action = np.where(pred >= THRESHOLD, 1, 0).astype(int)
        else:
            pred_df.action = 0
        env.predict(pred_df)
else:
    print('Training done!')